In [59]:
def extract_num(x):
    if x == '':
        return np.nan
    return re.search('\d+',x).group()
data = pd.DataFrame( data = df['cert_number'].apply(lambda x:extract_num(x)).dropna().values ,columns=['號碼'] )
data

,號碼
0,1028
1,1009
2,1017
3,1016
4,1023
...,...
17004,0209
17005,0539
17006,0426
17007,0572


# main

In [40]:
import csv 
import requests
import re 
import json 
import os

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import random
import pandas as pd
import numpy as np
from selenium.webdriver.support.ui import WebDriverWait       
from selenium.webdriver.common.by import By       
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [255]:
# crawldf.to_excel('律師資料.xlsx')

In [254]:
# crawldf = pd.DataFrame(index=crawldf.index)
crawldf

,109外檢證字第0003號,101臺檢證字第10004號,97臺檢證字第8000號,101臺檢證字第10009號,95臺檢證字第7000號,101臺檢證字第10006號,(69)臺檢證字第0009號,105臺檢證字第13000號,104臺檢證字第12000號,(69)臺檢證字第0001號,...,99臺檢證字第8890號,99臺檢證字第8895號,99臺檢證字第8892號,99臺檢證字第8906號,99臺檢證字第8900號,99臺檢證字第8907號,77台檢證935,台檢補證32,台檢補證53,台檢補證73
欄位名稱,,,,,,,,,,,,,,,,,,,,,
律師執業狀態,正常,正常,正常,正常,正常,正常,正常,正常,正常,正常,...,正常,正常,正常,正常,正常,正常,正常,正常,正常,正常
姓名,文魯彬(舊名 : 無),伍思樺(舊名 : 無),余若凡(舊名 : 無),吳俊宏(舊名 : 無),李承訓(舊名 : 無),李明勳(舊名 : 無),李新興(舊名 : 無),李蕙萱(舊名 : 無),李麗君(舊名 : 無),沈棱(舊名 : 無),...,洪旻郁(舊名 : 無),洪郡佑(舊名 : 無),陳昱澤(舊名 : 無),王全中(舊名 : 無),郭佳瑋(舊名 : 無),陳昭勳(舊名 : 無),陳廷献(舊名 : 無),王健(舊名 : 無),王昧爽(舊名 : 無),林宜君(舊名 : 陳林宜君)
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,,,,,,
英文姓名,Robin J.Winkler,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,,,,,,
性別,男,女,女,男,男,男,男,女,女,男,...,女,男,男,男,女,男,男,男,男,女
出生年份,西元1954年,民國75年,民國64年,民國70年,民國51年,民國73年,民國38年,民國75年,民國73年,民國26年,...,民國74年,民國67年,民國74年,民國55年,民國72年,民國68年,民國32年,民國3年,民國28年,民國54年
證書字號,109外檢證字第0003號,101臺檢證字第10004號,97臺檢證字第8000號,101臺檢證字第10009號,95臺檢證字第7000號,101臺檢證字第10006號,(69)臺檢證字第0009號,105臺檢證字第13000號,104臺檢證字第12000號,(69)臺檢證字第0001號,...,99臺檢證字第8890號,99臺檢證字第8895號,99臺檢證字第8892號,99臺檢證字第8906號,99臺檢證字第8900號,99臺檢證字第8907號,77台檢證935,台檢補證32,台檢補證53,台檢補證73
所屬公會,台北律師公會,台北律師公會,台北律師公會,台北律師公會,桃園律師公會,台北律師公會,台北律師公會,台北律師公會,桃園律師公會,台北律師公會,...,台北律師公會,台中律師公會,台中律師公會,台中律師公會,台北律師公會,台中律師公會,台中律師公會,台北律師公會,台北律師公會,台北律師公會
原資格國名/地區,美國亞利桑那州,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,,,,,,


## 搜索範圍:從000搜尋至999  + 證00至99 + 給證0至9 + 補證0至9
    2021/01最大號為15926
    全部範圍可能是000~999 + 證0~證9 + 證00~99

In [235]:
#main 1
chrome_path = 'D:/chromedriver.exe'#chromedriver放的位置
driver= webdriver.Chrome(chrome_path)#開啟瀏覽器
driver.get('https://lawyerbc.moj.gov.tw/')#輸入網址
driver.set_page_load_timeout(30)
driver.implicitly_wait(20)
WebDriverWait(driver, 60)
driver.set_script_timeout(30)
time.sleep(2)

In [253]:
#main 2
# 000到999
n1 = ['00'+str(x) for x in np.arange(0,10)]
n2 = ['0'+str(x) for x in np.arange(10,100)]
n3 = [str(x) for x in np.arange(100,1000)]
allnum = n1+n2+n3 
# 證00至99 + 給證0至9 + 補證0至9
n4 = ['證0'+str(x) for x in np.arange(0,10)]+['證'+str(x) for x in np.arange(10,100)]
n5 = ['給證'+str(x) for x in np.arange(0,10)]+['補證'+str(x) for x in np.arange(0,10)]
chinum=n4+n5

cut_point=None #被中斷後會存入當下的ite
restart_times=0 #中斷後重啟的次數
while restart_times <=3:
    try:
        # for item,num in enumerate(['158','159']): # 2021/01更新15787以上的號碼
        for ite,num in enumerate(allnum):#num輸入搜尋的字元
            if cut_point:
                if ite < cut_point:
                    continue
            driver.find_elements_by_class_name('el-input__inner')[0].clear()
            driver.find_elements_by_class_name('el-input__inner')[0].send_keys(num)  #輸入查詢字元
            time.sleep(0.5+random.random())
            #搜尋按鈕   
            element = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.XPATH,'/html/body/div/div/div[1]/div/div/div[1]/div/div/div[1]/button')))
            driver.execute_script("arguments[0].click();", element)
            time.sleep(2+random.random())
            page = driver.page_source
            soup = BeautifulSoup(page, 'lxml')
            #  if soup.find(class_ = 'el-message-box'): # 跳出"找不到符合條件之律師"畫面
            if not soup.find(class_='el-button el-button--default el-button--mini is-circle isLimit'): # 顯示50筆按鈕的tag
                driver.find_element_by_xpath('/html/body/div[3]/div/div[3]/button').click() #確定按鈕
                time.sleep(2+random.random())
                continue
            # 每頁顯示50筆數按鈕
            element = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.XPATH,'/html/body/div[1]/div[1]/div[1]/div/div/div[2]/div[2]/div[2]/div/div[2]/button[3]')))
            driver.execute_script("arguments[0].click();", element)
            time.sleep(1+random.random())
            page = driver.page_source
            soup = BeautifulSoup(page, 'lxml')
            lastpageNum = int(soup.find_all(class_='number')[-1].text) # 共有幾頁搜尋結果
            for p in range(lastpageNum):
                if p == 0: # 第一頁
                    pass
                else:
                    element = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.CLASS_NAME,'btn-next')))
                    driver.execute_script("arguments[0].click();", element) # 按下一頁
                    time.sleep(2+random.random())
                    page = driver.page_source
                    soup = BeautifulSoup(page, 'lxml')
                ### lineResults 是每一列搜尋結果
                lineResults = soup.find_all(class_ = 'el-tooltip lawyerList el-row is-justify-center is-align-middle el-row--flex')
                for i,line in enumerate(lineResults):
                    fullnum = line.find(class_='el-col el-col-8').text #律師證書字號
                    if fullnum not in {x for x in crawldf.columns}: #律師證書字號尚未收集到時
                    #     print(lineResult.find(class_='el-col el-col-8').text)
                        # 點擊一列搜尋結果
                        onoff = True
                        while onoff:
                            try:
                                element = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.XPATH,'/html/body/div[1]/div[1]/div[1]/div/div/div[2]/div[2]/div[3]/div[{}]/div[1]'.format(i+2))))
                                driver.execute_script("arguments[0].click();", element)
                                onoff = False
                            except TimeoutException: # selenium.common.exceptions.TimeoutException
                                print('cant click into',fullnum,end=',')
                                driver.refresh() #回到搜尋首頁
                                driver.find_elements_by_class_name('el-input__inner')[0].clear()
                                driver.find_elements_by_class_name('el-input__inner')[0].send_keys(num)  #重新輸入一樣的查詢字元
                                time.sleep(0.5+random.random())
                                #搜尋按鈕   
                                element = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.XPATH,'/html/body/div/div/div[1]/div/div/div[1]/div/div/div[1]/button')))
                                driver.execute_script("arguments[0].click();", element)
                                time.sleep(2+random.random())
                                # 每頁顯示50筆數按鈕
                                element = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.XPATH,'/html/body/div[1]/div[1]/div[1]/div/div/div[2]/div[2]/div[2]/div/div[2]/button[3]')))
                                driver.execute_script("arguments[0].click();", element)
                                time.sleep(1+random.random())
                                if p >0: #按回去該頁
                                    for cl in range(p):
                                        element = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.CLASS_NAME,'btn-next')))
                                        driver.execute_script("arguments[0].click();", element) # 按下一頁
                                        time.sleep(2+random.random())
                                time.sleep(4)
                        time.sleep(2.5+random.random())
                        page = driver.page_source
                        soup = BeautifulSoup(page, 'lxml')
                        cells=[]
                        if soup.find(class_='notfound'): # 點入個人資訊頁面後網頁顯示無此證號資料
                            cells.append(soup.find(class_='notfound').find('h3').text)
                            crawldf[fullnum] = cells+[np.nan]*19
                        else:
                            # 確認網頁有無完全載入完畢
                            onoff = True
                            while onoff:
                                # 有設 driver.set_page_load_timeout(30)後，執業狀態跟性別還都是空字串時，等待後再抓
                                if not soup.find(class_='el-col el-col-6 el-col-xs-18 el-col-sm-6 el-col-md-7'):
                                    print('"'+fullnum+'"',end=',')
                                    time.sleep(3)
                                    page = driver.page_source
                                    soup = BeautifulSoup(page, 'lxml')
                                elif soup.find(class_='el-col el-col-6 el-col-xs-18 el-col-sm-6 el-col-md-7').text == '' and soup.find(class_='el-col el-col-20 el-col-xs-18 el-col-sm-20 el-col-md-21').text == '':
                                    print('"'+fullnum+'"',end=',')
                                    time.sleep(3)
                                    page = driver.page_source
                                    soup = BeautifulSoup(page, 'lxml')
                                else: # 執業狀態跟性別至少一個有資訊
                                    onoff = False
                            #蒐集表格資料        
                            for i,tag in enumerate(soup.find(class_='lawyerInfo').find(class_='el-row').children): #表格裡的每一格
                                if i %2!=0: #基數格才是資訊，偶數格是欄位名稱
                                    cells.append(tag.text)
                            crawldf[fullnum] = cells[:-1]
                        driver.back() #回至搜尋結果列表
                        time.sleep(2+random.random())
                    else:
#                         print('exist-'+fullnum,end=',')
                        pass
                time.sleep(1+random.random())
    except: # 其他element錯誤、out of memory
        print('restart','ite='+ite,',num='+num)
        try: # 因如果是out of memory 便會error
            driver.close()
        except:
            pass
        driver= webdriver.Chrome(chrome_path)#開啟瀏覽器
        driver.get('https://lawyerbc.moj.gov.tw/')#輸入網址
        time.sleep(2)
        cut_point= ite
            

TypeError: can only concatenate str (not "int") to str

In [246]:
fullnum,i,ite,num,p

('77台檢證935', 0, 119, '補證9', 0)

## 調整欄位

In [284]:
data = crawldf.T
data.pop('證書字號')
data.pop(np.nan)
data.columns

Index(['律師執業狀態    ', '姓名', '英文姓名    ', '性別', '出生年份', '所屬公會', '原資格國名/地區',
       '執業許可證效期', '事務所名稱', '電子郵件', '電話', '地址', '懲戒記錄', '專業證照', '應迴避機關',
       '資料雜湊值', '發布日期', '更新日期'],
      dtype='object', name='欄位名稱')

In [285]:
data['網址'] = 'https://lawyerbc.moj.gov.tw/lawyer/'+ data['資料雜湊值']

In [304]:
import re
def cut(x):
    if type(x)==str:
        if re.search('\(舊.*\)',x): 
            x= x.replace(re.search('\(舊.*\)',x).group(),'')
    return x
data['姓名'] = data['姓名'].apply(lambda x: cut(x))


In [328]:
def minus1911(x):
    if type(x) == str:
        if '西元' in x or re.search('\d{4}',x):
            x = int(re.search('\d{4}',x).group())-1911
        elif '未提供' in x or '民國null年' in x:
            x = np.nan
        else:
            x = int(re.search('\d+',x).group())
    return x
data['出生民國年'] = data['出生年份'].apply(lambda x: minus1911(x) )

In [332]:
data.index.name= '證書字號'

In [335]:
data

欄位名稱,律師執業狀態,姓名,英文姓名,性別,出生年份,所屬公會,原資格國名/地區,執業許可證效期,事務所名稱,電子郵件,電話,地址,懲戒記錄,專業證照,應迴避機關,資料雜湊值,發布日期,更新日期,網址,出生民國年
證書字號,,,,,,,,,,,,,,,,,,,,
109外檢證字第0003號,正常,文魯彬,Robin J.Winkler,男,西元1954年,台北律師公會,美國亞利桑那州,109/03/18~112/03/17,社團法人台灣蠻野心足生態協會,rwinkler@wildatheart.org.tw,(02)2382-5789,100 台北市中正區懷寧街106號6樓之1,無,無,NaN,0x831c096b4a4a428b1a78ec4cd61e27545e408c48b0ec...,2020/11/30,2021/01/29,https://lawyerbc.moj.gov.tw/lawyer/0x831c096b4...,43.0
101臺檢證字第10004號,正常,伍思樺,NaN,女,民國75年,台北律師公會,NaN,101/09/07~,律宇國際商務法律事務所,無,(02)2771-9986分機17,10689 台北市大安區仁愛路四段107號8樓,無,無,NaN,0x8c7ee793f79c6c0caaf91590a245d559b69d3b7e2d4f...,2020/11/30,2021/01/29,https://lawyerbc.moj.gov.tw/lawyer/0x8c7ee793f...,75.0
97臺檢證字第8000號,正常,余若凡,NaN,女,民國64年,台北律師公會,NaN,097/08/07~,國際通商法律事務所,jo-fan.yu@bakermckenzie.com,(02)2712-6151,105 台北市松山區敦化北路168號15樓,無,無,NaN,0x41184f8013fae9c59b200e4c2c0d09f3325645394616...,2020/11/30,2021/01/29,https://lawyerbc.moj.gov.tw/lawyer/0x41184f801...,64.0
101臺檢證字第10009號,正常,吳俊宏,NaN,男,民國70年,台北律師公會,NaN,101/09/07~,瑞勤法律事務所,wolfwu77@yahoo.com.tw,(02)2558-8550,103 台北市大同區南京西路41號4樓之3,無,無,NaN,0xf73b74d0951fd6580ec735f854665ec6ee02e4761760...,2020/11/30,2021/01/29,https://lawyerbc.moj.gov.tw/lawyer/0xf73b74d09...,70.0
95臺檢證字第7000號,正常,李承訓,NaN,男,民國51年,桃園律師公會,NaN,095/07/20~,浩理法律事務所,holylawfirm@gmail.com,(03)3027092,330012桃園市桃園區大興西路２段６號五樓之１,無,無,NaN,0x2fe7945562d8fdbd7f58015f35cf390f429a60c0512b...,2020/11/30,2021/01/29,https://lawyerbc.moj.gov.tw/lawyer/0x2fe794556...,51.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99臺檢證字第8907號,正常,陳昭勳,,男,民國68年,台中律師公會,,099/12/31~,詮誠國際法律事務所,lcobychen09@gmail.com,04-22626572#331,402 台中市南區忠明南路758號16樓,無,無,,0x59d56f2859fd390eab49a60d354c86ef21eabcabcab6...,2020/11/30,2021/01/29,https://lawyerbc.moj.gov.tw/lawyer/0x59d56f285...,68.0
77台檢證935,正常,陳廷献,,男,民國32年,台中律師公會,,077/10/03~,宏全法律事務所,cts4001@yahoo.com.tw,0921-057586,臺中市西區繼光街4之1號,無,無,,0x5ee897e98448a7f72b08dbe8e12a1d2c095008c59ef3...,2020/11/30,2021/01/29,https://lawyerbc.moj.gov.tw/lawyer/0x5ee897e98...,32.0
台檢補證32,正常,王健,,男,民國3年,台北律師公會,,076/05/11~,秉公法律事務所,binggong@ms36.hinet.net,(02)2541-0424,104 台北市中山區松江路146號5樓之3,無,無,,0x92854c8f0151fce64e0ed2fde95b47295ad0044dbc89...,2020/11/30,2021/01/29,https://lawyerbc.moj.gov.tw/lawyer/0x92854c8f0...,3.0


In [336]:
data.to_excel('律師資料.xlsx')